# 1. Tiến hành đọc hết tất cả các review của các sản phẩm vào một dataframe duy nhất

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import modules.utils as Utils

In [3]:
urls = [
    "../00. Data Scraping/data/shopee_reviews/men/",
    "../00. Data Scraping/data/shopee_reviews/women/",
    "../00. Data Scraping/data/shopee_reviews/more/",
    "../00. Data Scraping/data/shopee_reviews/mor/"
]

In [4]:
raw_data = pd.DataFrame(columns=['comment', 'rating'])

In [5]:
for url in urls:
    products = Utils.getFiles(url)
    for product in products:
        df = pd.read_csv(url + product)
        raw_data = pd.concat([raw_data, df], axis=0).reset_index(drop=True)

In [6]:
raw_data

,comment,rating
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",5
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",5
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",5
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,5
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,5
...,...,...
278154,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,5
278155,Dởm,5
278156,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,5
278157,Hàng đẹp ạ😍,5


> **Nhận xét**
> * Có tổng cộng 278159 reviews

🚩 Ghi ra file csv

In [7]:
raw_data.to_csv("./data/raw_data.csv", index=False)

<hr>

# 2. Tìm hiểu dữ liệu

🚩 Đánh nhãn cho dữ liệu, nếu rating của khách hàng là:
* $\leq 2$: Tiêu cực
* $\in [3, 4]$: Bình thường
* $= 5$: Tích cực

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import pandas as pd

In [10]:
def rating(pScore):
    if pScore <= 2:
        return -1
    
    if pScore <= 4:
        return 0
    
    return 1

In [11]:
raw_data = pd.read_csv("./data/raw_data.csv")

In [12]:
raw_data['label'] = raw_data['rating'].apply(lambda score: rating(score))

In [13]:
raw_data

,comment,rating,label
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",5,1
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",5,1
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",5,1
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,5,1
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,5,1
...,...,...,...
278154,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,5,1
278155,Dởm,5,1
278156,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,5,1
278157,Hàng đẹp ạ😍,5,1


🚩 Đếm nhóm theo `label` và đếm xem có bao nhiêu phần từ thuộc nhóm đó

In [14]:
raw_data.groupby('label').agg('count')

,comment,rating
label,,
-1,3764,3764
0,13840,13840
1,260555,260555


> **Nhận xét:**
> * Có sự chênh lệch lớn giữa các mẫu

🚩 Lưu data

In [15]:
raw_data.to_csv("./data/labeled_data.csv", index=False)

# 3. Tiền xử lí dữ liệu

* Các bước cần quan tâm:
  * Extract emoji
  * Chuẩn hết về chữ thường
  * Remove các mẫu chứa các URL (khả năng cao là quảng cáo)
  * Loại bỏ dấu câu và các kí tự đặc biệt.
  * Xử lí các trường hợp từ láy láy âm tiết (ví dụ: ngonnnnn quáááááááááááá điiiiiiiiiii)
  * Chuẩn hóa các từ viết tắt.
  * Tác từ (underthesea hoặc VnCoreNLP)

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import pandas as pd
import modules.utils as Utils
import pickle

In [18]:
data = pd.read_csv("./data/labeled_data.csv")

🚩 Extract emoji

In [19]:
data['emoji'] = data['comment'].apply(lambda comment: Utils.extract_emoji(comment))

In [20]:
data

,comment,rating,label,emoji
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",5,1,[]
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",5,1,[]
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",5,1,[]
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,5,1,[]
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,5,1,[]
...,...,...,...,...
278154,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,5,1,[]
278155,Dởm,5,1,[]
278156,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,5,1,[]
278157,Hàng đẹp ạ😍,5,1,[😍]


🚩 Chuẩn hóa `comment` về chữ thường

In [21]:
data['lowercase_comment'] = data['comment'].apply(lambda comment: comment.lower())

In [22]:
data

,comment,rating,label,emoji,lowercase_comment
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",5,1,[],"lần đầu mua tại shop nhưng rất ưng ý, vải mặc ..."
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",5,1,[],"giao hàng nhanh, shop xịn xò đặt hàng xong là ..."
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",5,1,[],"size vừa vặn luôn, shop mall mà hàng chất lượn..."
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,5,1,[],sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,5,1,[],chất lượng sản phẩm tuyệt vời giao hàng nhanh ...
...,...,...,...,...,...
278154,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,5,1,[],đẹp lắm lun mọi ngừ ơi . hdhhechhjhshxhdhhxhxh...
278155,Dởm,5,1,[],dởm
278156,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,5,1,[],mua 12 món giao đủ. vừa rẻ vừa chất lượng rất ...
278157,Hàng đẹp ạ😍,5,1,[😍],hàng đẹp ạ😍


🚩 Đánh dấu các mẫu chứa các URL (khả năng cao là quảng cáo)

In [23]:
data['contain_url'] = data['lowercase_comment'].apply(lambda comment: 1 if Utils.containsURL(comment) == True else 0)

In [24]:
data

,comment,rating,label,emoji,lowercase_comment,contain_url
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",5,1,[],"lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",0
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",5,1,[],"giao hàng nhanh, shop xịn xò đặt hàng xong là ...",0
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",5,1,[],"size vừa vặn luôn, shop mall mà hàng chất lượn...",0
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,5,1,[],sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,0
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,5,1,[],chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,0
...,...,...,...,...,...,...
278154,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,5,1,[],đẹp lắm lun mọi ngừ ơi . hdhhechhjhshxhdhhxhxh...,0
278155,Dởm,5,1,[],dởm,0
278156,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,5,1,[],mua 12 món giao đủ. vừa rẻ vừa chất lượng rất ...,0
278157,Hàng đẹp ạ😍,5,1,[😍],hàng đẹp ạ😍,0


In [25]:
data.loc[data['contain_url'] == 1]

,comment,rating,label,emoji,lowercase_comment,contain_url
22,Mình bán Khăn Giấy Ướt CÓ GIẤY KIỂM NGHIỆM 80 ...,5,1,[],mình bán khăn giấy ướt có giấy kiểm nghiệm 80 ...,1
100,"(MOBIFONE) MOI LUC MOI NOI, KHONG LO DICH BENH...",5,1,[],"(mobifone) moi luc moi noi, khong lo dich benh...",1
115,[TB] TẶNG 20% GIÁ TRỊ TẤT CẢ THẺ NẠP trong ng...,5,1,[],[tb] tặng 20% giá trị tất cả thẻ nạp trong ng...,1
163,Quy khach duoc cong them 500 diem Viettel++ bo...,5,1,[],quy khach duoc cong them 500 diem viettel++ bo...,1
596,https://facebook.com/103325964856698/posts/273...,5,1,[],https://facebook.com/103325964856698/posts/273...,1
...,...,...,...,...,...,...
276515,https://www.facebook.com/groups/PUBGMobileGrou...,5,1,[],https://www.facebook.com/groups/pubgmobilegrou...,1
277197,☘️☘️☘️☘️☘️\nxin lỗi đã làm phiền và cảm ơn đã ...,5,1,"[💖, 💕, ☘️]",☘️☘️☘️☘️☘️\nxin lỗi đã làm phiền và cảm ơn đã ...,1
277389,"Siêu dễ thương,bé rất thích. Em bán son lì khô...",5,1,[],"siêu dễ thương,bé rất thích. em bán son lì khô...",1
277735,https://www.facebook.com/1525957184368292/post...,5,1,[],https://www.facebook.com/1525957184368292/post...,1


> **Nhận xét**:
> * Có đến 570 mẫu chứa khả năng cao là có quảng cáo

🚩 Loại bỏ dấu câu và các kí tự đặc biệt và số

In [26]:
data['lowercase_comment'] = data['lowercase_comment'].apply(lambda comment: Utils.removeSpecialCharacters(comment))

In [27]:
data

,comment,rating,label,emoji,lowercase_comment,contain_url
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",5,1,[],lần đầu mua tại shop nhưng rất ưng vải mặc mát...,0
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",5,1,[],giao hàng nhanh shop xịn xò đặt hàng xong là c...,0
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",5,1,[],size vừa vặn luôn shop mall mà hàng chất lượng...,0
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,5,1,[],sản phẩm chất lượng đóng gói kĩ shop nhiệt tìn...,0
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,5,1,[],chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,0
...,...,...,...,...,...,...
278154,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,5,1,[],đẹp lắm lun mọi ngừ ơi hdhhechhjhshxhdhhxhxhdh...,0
278155,Dởm,5,1,[],dởm,0
278156,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,5,1,[],mua món giao đủ vừa rẻ vừa chất lượng rất thíc...,0
278157,Hàng đẹp ạ😍,5,1,[😍],hàng đẹp ạ,0


🚩 Chuẩn lại các từ bị duplicate kí tự lên (ví dụ: "anh nhớớớớớớớớớớ emmmmmmmmmmmmmmmmmmmmm quááááááááááááááá" thành "anh nhớ em quá")

In [28]:
data['lowercase_comment'] = data['lowercase_comment'].apply(lambda comment: Utils.removeDupplicateLetters(comment))

In [29]:
data

,comment,rating,label,emoji,lowercase_comment,contain_url
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",5,1,[],lần đầu mua tại shop nhưng rất ưng vải mặc mát...,0
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",5,1,[],giao hàng nhanh shop xịn xò đặt hàng xong là c...,0
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",5,1,[],size vừa vặn luôn shop mal mà hàng chất lượng ...,0
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,5,1,[],sản phẩm chất lượng đóng gói kĩ shop nhiệt tìn...,0
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,5,1,[],chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,0
...,...,...,...,...,...,...
278154,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,5,1,[],đẹp lắm lun mọi ngừ ơi hdhechjhshxhdhxhxhdh hs...,0
278155,Dởm,5,1,[],dởm,0
278156,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,5,1,[],mua món giao đủ vừa rẻ vừa chất lượng rất thíc...,0
278157,Hàng đẹp ạ😍,5,1,[😍],hàng đẹp ạ,0


🚩 Chuẩn lại các từ bị viết tắt (ví dụ: "hk" thành "không")

In [30]:
vietnamese_acronym_words = None

with open('./legacy/vietnamese_acronym_words.pickle', 'rb') as reader:
    vietnamese_acronym_words = pickle.load(reader)

In [31]:
data['lowercase_comment'] = data['lowercase_comment'].apply(lambda comment: Utils.fixAcronymWords(vietnamese_acronym_words, comment))

In [32]:
data

,comment,rating,label,emoji,lowercase_comment,contain_url
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",5,1,[],lần đầu mua tại shop nhưng rất ưng vải mặc mát...,0
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",5,1,[],giao hàng nhanh shop xịn xò đặt hàng xong là c...,0
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",5,1,[],size vừa vặn luôn shop mal mà hàng chất lượng ...,0
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,5,1,[],sản phẩm chất lượng đóng gói kĩ shop nhiệt tìn...,0
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,5,1,[],chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,0
...,...,...,...,...,...,...
278154,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,5,1,[],đẹp lắm luôn mọi người ơi hdhechjhshxhdhxhxhdh...,0
278155,Dởm,5,1,[],dởm,0
278156,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,5,1,[],mua món giao đủ vừa rẻ vừa chất lượng rất thíc...,0
278157,Hàng đẹp ạ😍,5,1,[😍],hàng đẹp ạ,0


🚩 Loại bỏ các từ rác ra khỏi comment, (ví dụ: "asfsfskjfhfjkfhf", "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa")

In [33]:
vietnamese_words = None

with open('./legacy/vietnamese_words.pickle', 'rb') as reader:
    vietnamese_words = pickle.load(reader)

In [34]:
keys = list(vietnamese_words.keys())
print(keys[:5])

['a', 'ba', 'boa', 'ca', 'cha']


In [35]:
data['lowercase_comment'] = data['lowercase_comment'].apply(lambda comment: Utils.removeGibbish(vietnamese_words, comment))

In [36]:
data

,comment,rating,label,emoji,lowercase_comment,contain_url
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",5,1,[],lần đầu mua tại shop nhưng rất ưng vải mặc mát...,0
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",5,1,[],giao hàng nhanh shop đặt hàng xong là chủ động...,0
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",5,1,[],size vừa vặn luôn shop mà hàng chất lượng giá ...,0
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,5,1,[],sản phẩm chất lượng đóng gói kĩ shop nhiệt tìn...,0
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,5,1,[],chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,0
...,...,...,...,...,...,...
278154,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,5,1,[],đẹp lắm luôn mọi người ơi học,0
278155,Dởm,5,1,[],dởm,0
278156,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,5,1,[],mua món giao đủ vừa rẻ vừa chất lượng rất thíc...,0
278157,Hàng đẹp ạ😍,5,1,[😍],hàng đẹp ạ,0


🚩 Ghép cột `lowercase_comment` với `emoji` lại thành cột `standard_comment`

In [37]:
data['standard_comment'] = data.apply(lambda row: Utils.combineCommentAnnEmoji(row['lowercase_comment'], row['emoji']), axis=1)

In [38]:
data

,comment,rating,label,emoji,lowercase_comment,contain_url,standard_comment
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",5,1,[],lần đầu mua tại shop nhưng rất ưng vải mặc mát...,0,lần đầu mua tại shop nhưng rất ưng vải mặc mát...
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",5,1,[],giao hàng nhanh shop đặt hàng xong là chủ động...,0,giao hàng nhanh shop đặt hàng xong là chủ động...
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",5,1,[],size vừa vặn luôn shop mà hàng chất lượng giá ...,0,size vừa vặn luôn shop mà hàng chất lượng giá ...
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,5,1,[],sản phẩm chất lượng đóng gói kĩ shop nhiệt tìn...,0,sản phẩm chất lượng đóng gói kĩ shop nhiệt tìn...
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,5,1,[],chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,0,chất lượng sản phẩm tuyệt vời giao hàng nhanh ...
...,...,...,...,...,...,...,...
278154,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,5,1,[],đẹp lắm luôn mọi người ơi học,0,đẹp lắm luôn mọi người ơi học
278155,Dởm,5,1,[],dởm,0,dởm
278156,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,5,1,[],mua món giao đủ vừa rẻ vừa chất lượng rất thíc...,0,mua món giao đủ vừa rẻ vừa chất lượng rất thíc...
278157,Hàng đẹp ạ😍,5,1,[😍],hàng đẹp ạ,0,hàng đẹp ạ 😍


🚩 Xóa các sample có `standard_comment` duplicate

In [40]:
data = data.drop_duplicates(subset=['standard_comment'])

In [41]:
data

,comment,rating,label,emoji,lowercase_comment,contain_url,standard_comment
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",5,1,[],lần đầu mua tại shop nhưng rất ưng vải mặc mát...,0,lần đầu mua tại shop nhưng rất ưng vải mặc mát...
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",5,1,[],giao hàng nhanh shop đặt hàng xong là chủ động...,0,giao hàng nhanh shop đặt hàng xong là chủ động...
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",5,1,[],size vừa vặn luôn shop mà hàng chất lượng giá ...,0,size vừa vặn luôn shop mà hàng chất lượng giá ...
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,5,1,[],sản phẩm chất lượng đóng gói kĩ shop nhiệt tìn...,0,sản phẩm chất lượng đóng gói kĩ shop nhiệt tìn...
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,5,1,[],chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,0,chất lượng sản phẩm tuyệt vời giao hàng nhanh ...
...,...,...,...,...,...,...,...
278153,"Rất dễ thương, đẹp",5,1,[],rất dễ thương đẹp,0,rất dễ thương đẹp
278154,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,5,1,[],đẹp lắm luôn mọi người ơi học,0,đẹp lắm luôn mọi người ơi học
278155,Dởm,5,1,[],dởm,0,dởm
278156,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,5,1,[],mua món giao đủ vừa rẻ vừa chất lượng rất thíc...,0,mua món giao đủ vừa rẻ vừa chất lượng rất thíc...


In [43]:
data.groupby('label').count()['standard_comment']

label
-1      3487
 0     11931
 1    205899
Name: standard_comment, dtype: int64

> **Nhận xét**:
> * -1: tiêu cực, 0 bình thường và 1 là tích cực

🚩 Ghi ra file 

In [44]:
data.to_csv("./data/standard_data.csv", index=False)